In [33]:
import pandas as pd
import gensim
from gensim import corpora, models
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import wordnet as wn
import pickle

In [34]:
# Read the data being used to create the model
df = pd.read_csv('FinalData/qdata.csv', encoding='utf-8')
df = df[df['Text'].notna()]

print(df.head())

dict = df.to_dict('index')


   Unnamed: 0       Id  AnswerCount             CreationDate  OwnerUserId  \
0           0  1786915            3  2009-11-23 23:54:33.523       178189   
1           1  1786917            2  2009-11-23 23:55:14.417       160359   
2           2  1786919            0  2009-11-23 23:55:34.790       102215   
3           3     2809            5  2008-08-05 20:33:21.757          437   
4           4    19995           18  2008-08-21 14:04:55.540          184   

   Score  ViewCount                                               Text    neg  \
0      0       1379  i was wondering how people are handling the la...  0.010   
1     55      14451  i have a project that adds some extensibility ...  0.000   
2      1        397  i am a novice to eclipse cdt and its ui for cv...  0.058   
3     11       2644  the table does not have a last updated field a...  0.075   
4    134      73348  of course i am aware of ajax, but the problem ...  0.070   

     neu    pos   Tags_y  
0  0.870  0.120        

In [35]:
def tokenizeText(text):
    return word_tokenize(text)

In [36]:
def removeFirstLastThree(text):
    text = text[3:]
    text = text[:len(text)-3]
    return text

In [37]:
def toLowerCase(text):
    text = [word.lower() for word in text]
    return text

In [38]:
def removeStopWords(text):
    stop_words = set(stopwords.words("english"))
    filtered_words = []

    for w in text:
        if w not in stop_words:
            filtered_words.append(w)
            
    return filtered_words

In [39]:
def applyPStemmer(text):
    ps = PorterStemmer()

    stemmed_words=[]
    for w in text:
        stemmed_words.append(ps.stem(w))

    return stemmed_words

In [41]:

def get_lemma(text):
    words = []
    for word in text:
        lemma = wn.morphy(word)
        if (len(word) <= 2 or len(word) >= 15 or word == 'code' or word.isnumeric() or word == 'gt' or word == 'lt' or word =='quot' or word == 'pre' or word == 'amp'):
            continue 
        elif lemma is None or word == lemma:
            words.append(word)
        else:
            words.append(lemma)
    return words
from nltk.stem.wordnet import WordNetLemmatizer

def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

In [42]:
# Process LDA model
text_tokens = []
i = 0
for key in dict:
    tokens = tokenizeText(dict[key]['Text'])
    tokens = removeFirstLastThree(tokens)
    tokens = toLowerCase(tokens)
    tokens = removeStopWords(tokens)
    tokens = applyPStemmer(tokens)
    tokens = get_lemma(tokens)

    

    if i < 5:
        print(tokens)
    text_tokens.append(tokens)
    i += 1

['peopl', 'handl', 'lack', 'enum', 'support', 'entiti', 'framework', 'deal', 'wcf', 'servic', 'common', 'practic', 'seem', 'declar', 'getter', 'setter', 'entiti', 'properti', 'privat', 'creat', 'partial', 'class', 'entiti', 'declar', 'enum', 'properti', 'fine', 'dandi', 'pose', 'issu', 'wcf', 'servic', 'need', 'enum', 'definit', 'client', 'side', 'want', 'happen', 'automat', 'proxi', 'gener', 'add', 'datamemb', 'attribut', 'enum', 'properti', 'share', 'class', 'includ', 'serial', 'process', 'howev', 'call', 'wcf', 'method', 'serial', 'int', 'enum', 'properti', 'thu', 'carri', 'redund', 'data', 'increas', 'messag', 'size', 'best', 'way', 'deal', 'peopl', 'handl', 'scenario']
['project', 'add', 'extens', 'anoth', 'applic', 'api', 'howev', 'want', 'abl', 'use', 'project', 'multipl', 'version', 'applic', 'howev', 'version', 'applic', 'requir', 'refer', 'proper', 'assembl', 'version', 'softwar', 'load', 'assembl', 'gac', 'even', 'could', 'specifi', 'version', 'assembl', 'use', 'base', 'buil

In [43]:
dict = corpora.Dictionary(text_tokens)
dict.save('dictionary.gensim')

bagOfWords = [dict.doc2bow(t) for t in text_tokens]
pickle.dump(bagOfWords, open('models/corpus.pkl', 'wb'))



Dictionary(133388 unique tokens: ['add', 'attribut', 'automat', 'best', 'call']...)


In [44]:
NUM_TOPICS = 25
NUM_WORDS = 2

# Commenting to avoid overwriting existing saved model
# ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
# ldamodel.save('models/lda_model.model')

ldamodel =  models.LdaModel.load('models/lda_model.model')
topics = ldamodel.print_topics(num_words = NUM_WORDS)


In [45]:
for topic in topics:
    print(topic)

(12, '0.192*"self" + 0.026*"let"')
(8, '0.118*"file" + 0.038*"line"')
(17, '0.115*"image" + 0.046*"category"')
(16, '0.107*"div" + 0.103*"class"')
(10, '0.084*"string" + 0.075*"public"')
(22, '0.058*"int" + 0.034*"array"')
(23, '0.237*"app" + 0.076*"tag"')
(15, '0.037*"version" + 0.030*"build"')
(14, '0.139*"property" + 0.047*"binding"')
(7, '0.067*"function" + 0.057*"var"')
(24, '0.022*"strong" + 0.019*"like"')
(19, '0.102*"http" + 0.073*"com"')
(18, '0.072*"stack" + 0.070*"http"')
(4, '0.032*"server" + 0.028*"test"')
(11, '0.192*"android" + 0.049*"view"')
(21, '0.115*"model" + 0.079*"date"')
(9, '0.033*"task" + 0.032*"system"')
(6, '0.048*"row" + 0.046*"list"')
(13, '0.066*"product" + 0.059*"query"')
(1, '0.065*"width" + 0.063*"color"')


In [46]:
new_doc = df.iloc[[4]]
print(new_doc)

text = 'when i click on a video, it is redirecting back to homepage, youtube clone'


# tokens = prepare_text_for_lda(dict[key]['Body'])
tokens = tokenizeText(text)
tokens = removeFirstLastThree(tokens)
tokens = toLowerCase(tokens)
tokens = removeStopWords(tokens)
tokens = applyPStemmer(tokens)
tokens = get_lemma(tokens)

print(tokens)


new_doc_bow = dict.doc2bow(tokens)
# print(new_doc_bow)
# print(ldamodel.get_document_topics(new_doc_bow))
# new_doc_bow = dictionary.doc2bow(new_doc)
# print(new_doc_bow)
print(ldamodel.get_document_topics(new_doc_bow))

   Unnamed: 0     Id  AnswerCount             CreationDate  OwnerUserId  \
4           4  19995           18  2008-08-21 14:04:55.540          184   

   Score  ViewCount                                               Text   neg  \
4    134      73348  of course i am aware of ajax, but the problem ...  0.07   

     neu    pos Tags_y  
4  0.853  0.076   ajax  
['video', 'redirect', 'back', 'homepag']
[(0, 0.010001341), (1, 0.010001341), (2, 0.010001341), (3, 0.010001341), (4, 0.010001341), (5, 0.010001341), (6, 0.010001341), (7, 0.010001341), (8, 0.010001341), (9, 0.010001341), (10, 0.010001341), (11, 0.010001341), (12, 0.010001341), (13, 0.010001341), (14, 0.010001341), (15, 0.010001341), (16, 0.010001341), (17, 0.010001341), (18, 0.010001341), (19, 0.010001341), (20, 0.010001341), (21, 0.010001341), (22, 0.42947686), (23, 0.010001341), (24, 0.3404923)]


In [47]:
loaded_model = gensim.models.ldamodel.LdaModel.load('models/lda_model.model')
print(loaded_model.get_document_topics(new_doc_bow))

print('\n')

topics = loaded_model.print_topics(num_words=2)
for topic in topics:
    print(topic)


[(0, 0.010001341), (1, 0.010001341), (2, 0.010001341), (3, 0.010001341), (4, 0.010001341), (5, 0.010001341), (6, 0.010001341), (7, 0.010001341), (8, 0.010001341), (9, 0.010001341), (10, 0.010001341), (11, 0.010001341), (12, 0.010001341), (13, 0.010001341), (14, 0.010001341), (15, 0.010001341), (16, 0.010001341), (17, 0.010001341), (18, 0.010001341), (19, 0.010001341), (20, 0.010001341), (21, 0.010001341), (22, 0.4296849), (23, 0.010001341), (24, 0.34028426)]


(19, '0.102*"http" + 0.073*"com"')
(18, '0.072*"stack" + 0.070*"http"')
(17, '0.115*"image" + 0.046*"category"')
(1, '0.065*"width" + 0.063*"color"')
(21, '0.115*"model" + 0.079*"date"')
(9, '0.033*"task" + 0.032*"system"')
(16, '0.107*"div" + 0.103*"class"')
(8, '0.118*"file" + 0.038*"line"')
(20, '0.057*"user" + 0.028*"request"')
(4, '0.032*"server" + 0.028*"test"')
(23, '0.237*"app" + 0.076*"tag"')
(2, '0.091*"name" + 0.062*"type"')
(0, '0.100*"php" + 0.074*"lib"')
(10, '0.084*"string" + 0.075*"public"')
(22, '0.058*"int" + 0.